In [1]:
from dustpy import Simulation
from dustpy import plot
from dustpy import constants as c
import numpy as np
import matplotlib.pyplot as plt

In [2]:
s = Simulation()

In [3]:
s.ini.dust.allowDriftingParticles = True
s.ini.grid.Nr = 5
s.ini.grid.rmin = 1. * c.au
s.ini.grid.rmax = 10. * c.au
s.ini.grid.mmin = 1.e0
s.ini.grid.mmax = 1.e6

In [4]:
s.initialize()

In [5]:
s.setdustintegrator()

Setting dust integrator
    scheme: explicit
    method: cash-karp


In [6]:
s.integrator.instructions

[Instruction (Dust: explicit 5th-order adaptive Cash-Karp method),
 Instruction (Gas: implicit 1st-order direct solver)]

In [7]:
del(s.integrator.instructions[1])

In [8]:
s.integrator.instructions

[Instruction (Dust: explicit 5th-order adaptive Cash-Karp method)]

In [9]:
s.gas.S.tot[...] = 0.
s.gas.S.tot.updater = None

In [10]:
s.dust.v.rad[...] = 0.
s.dust.v.rad.updater = None

In [11]:
s.dust.p.frag[...] = 0.
s.dust.p.frag.updater = None
s.dust.p.stick[...] = 1.
s.dust.p.stick.updater = None

In [12]:
s.dust.kernel[...] = 1.
s.dust.kernel.updater = None

In [13]:
s.t.snapshots = np.logspace(0., 5., 6)

In [14]:
s.writer.datadir = "kernel"
s.writer.overwrite = True

In [15]:
s.dust.Sigma[...] = 0.
s.dust.Sigma[:, 0] = 1.

In [16]:
s.update()

In [17]:
s.run()


DustPy v0.5.1

Documentation: https://stammler.github.io/dustpy/
PyPI:          https://pypi.org/project/dustpy/
GitHub:        https://github.com/stammler/dustpy/

Please read README.md on the GitHub repository for
information about the Terms of Usage.

Checking for mass conservation...

    - Sticking:
        max. rel. error:  5.49e-15
        for particle collision
            m[29] =  1.39e+04 g    with
            m[31] =  2.68e+04 g
    - Full fragmentation:
        max. rel. error:  3.33e-16
        for particle collision
            m[31] =  2.68e+04 g    with
            m[34] =  7.20e+04 g
    - Cratering:
        max. rel. error:  4.44e-16
        for particle collision
            m[21] =  1.00e+03 g    with
            m[29] =  1.39e+04 g

Writing file kernel/data0000.hdf5
Writing dump file kernel/frame.dmp


StopIteration: Maximum number of integration attempts exceeded.

In [ ]:
SigmaDust = s.writer.read.sequence("dust.Sigma")
t = s.writer.read.sequence("t")

In [ ]:
A = np.mean(s.grid.m[1:]/s.grid.m[:-1])
B = 2 * (A-1) / (A+1)
sigD = SigmaDust / B

In [ ]:
def solution_constant_kernel(t, m, N0):
    a = 0.5 * N0
    return N0 / m[0] * 1./(a*t)**2 * np.exp(-(m/m[0]-1)/(a*t))

In [ ]:
fig = plt.figure(dpi=150)
ax = fig.add_subplot(111)
for i in range(2, len(t)):
    cstr = "C" + str(i-2)
    ax.loglog(s.grid.m, sigD[i, 3, :], "-", lw=1, c=cstr, label="t = {:3.1f}".format(t[i]))
    ax.loglog(s.grid.m, s.grid.m**2*solution_constant_kernel(t[i], s.grid.m, 1.), "--", lw=1, c=cstr)
ax.legend()
ax.set_ylim(1.e-6, 1.e3)
ax.set_xlabel(r"$m$")
ax.set_ylabel(r"$m^2 f\left(m\right)$")
ax.set_title("Analytical Kernel: constant")
fig.tight_layout()
fig.savefig("analytical_kernel.png")